In [1]:
#this is deep learning section diff from the machine learning...
#here we use human like neural structure to build model for classification and regression...
#also process of data cleaning and other are same but model building and libraries are diffrent...

In [2]:
import tensorflow as tf#important library for deep learning...

In [3]:
import numpy as np
import pandas as pd
import kerastuner as kt#library for hyperparameter tuning

C:\Users\91916\AppData\Local\Temp\ipykernel_18620\3907407629.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt#library for hyperparameter tuning


In [4]:
df=pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
#check for null values and dtypes

In [6]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [7]:
df.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

In [8]:
#no null values and no object type data so lets move forward by scaling values to -1 to 1 by mon max scaler

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scale=MinMaxScaler()

In [11]:
#divide dataset into x and y

In [12]:
x=df.iloc[:,:-1].values#all column except last target col
y=df.iloc[:,-1].values#only target col in y

In [13]:
#scaling x data...

In [16]:
x_scaled=scale.fit_transform(x)

In [17]:
x_scaled.ndim

2

In [18]:
#now,....

In [19]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.3,random_state=1)

In [20]:
#now import libraries from tensorflow that need...

In [21]:
from tensorflow import keras

In [22]:
from keras import Sequential
from keras.layers import Dropout,Dense

In [23]:
#from tensorflow import keras and from keras import sequential,dense and dropout

In [24]:
#building model without hyperparametr tuning...

In [27]:
model=Sequential()
model.add(Dense(32,activation='relu',input_shape=(8,)))#input layer....
model.add(Dense(1,activation='sigmoid'))#last op layer
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

In [28]:
#making model fit on the data...

In [32]:
model.fit(x_train,y_train,batch_size=32,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
17/17 [==============================] - 0s 4ms/step - loss: 0.4627 - accuracy: 0.7821 - val_loss: 0.4455 - val_accuracy: 0.7922
Epoch 2/100
17/17 [==============================] - 0s 3ms/step - loss: 0.4625 - accuracy: 0.7858 - val_loss: 0.4444 - val_accuracy: 0.7879
Epoch 3/100
17/17 [==============================] - 0s 2ms/step - loss: 0.4616 - accuracy: 0.7877 - val_loss: 0.4463 - val_accuracy: 0.7965
Epoch 4/100
17/17 [==============================] - 0s 3ms/step - loss: 0.4621 - accuracy: 0.7821 - val_loss: 0.4459 - val_accuracy: 0.7965
Epoch 5/100
17/17 [==============================] - 0s 3ms/step - loss: 0.4617 - accuracy: 0.7877 - val_loss: 0.4444 - val_accuracy: 0.7965
Epoch 6/100
17/17 [==============================] - 0s 3ms/step - loss: 0.4619 - accuracy: 0.7840 - val_loss: 0.4455 - val_accuracy: 0.7965
Epoch 7/100
17/17 [==============================] - 0s 3ms/step - loss: 0.4621 - accuracy: 0.7858 - val_loss: 0.4451 - val_accuracy: 0.7922
Epoch 8/100
1

In [33]:
#now with hyperparametr tuning...

In [34]:
def build_model(hp):
  model=Sequential()
  counter=0
  for i in range(hp.Int("num_layers",min_value=1,max_value=10)):#no of hidden layers from 1 to 9...
    if counter==0:
      model.add(
          Dense(
              units=hp.Int("units"+str(i),min_value=8,max_value=128,step=8), #no of neurons in 1st hidden layer....
              activation=hp.Choice('Activation'+str(i),values=['relu','tanh','sigmoid']),#best optimizer from no of optimizers
              input_dim=8)
      )
      
    else:
      model.add(
          Dense(
              units=hp.Int("units"+str(i),min_value=8,max_value=128,step=8),
              activation=hp.Choice('Activation'+str(i),values=['relu','tanh','sigmoid']),
              )
      )
    counter+=1
  model.add(Dense(1,activation='sigmoid')) #designing output model.....
  optimizer=hp.Choice("optimizer",values=['adam','rmsprop','adadelta','sgd','nadam']) #optimizer for output layer....
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy']) #now compiling model...
  return model


In [35]:
tuner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=3,directory='tuning',project_name='version4')

INFO:tensorflow:Reloading Oracle from existing project tuning\version4\oracle.json
INFO:tensorflow:Reloading Tuner from tuning\version4\tuner0.json


In [36]:
#now passing data...

In [37]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

INFO:tensorflow:Oracle triggered exit


In [38]:
#getting best model values 

In [40]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units0': 112,
 'Activation0': 'tanh',
 'optimizer': 'adam',
 'units1': 80,
 'Activation1': 'sigmoid',
 'units2': 24,
 'Activation2': 'tanh',
 'units3': 64,
 'Activation3': 'sigmoid',
 'units4': 24,
 'Activation4': 'relu',
 'units5': 72,
 'Activation5': 'tanh',
 'units6': 80,
 'Activation6': 'tanh',
 'units7': 120,
 'Activation7': 'sigmoid',
 'units8': 72,
 'Activation8': 'relu',
 'units9': 8,
 'Activation9': 'relu'}

In [41]:
#now storing model with best parameter in 0 index...

In [42]:
model1=tuner.get_best_models(num_models=1)[0]

In [43]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 112)               1008      
                                                                 
 dense_1 (Dense)             (None, 80)                9040      
                                                                 
 dense_2 (Dense)             (None, 24)                1944      
                                                                 
 dense_3 (Dense)             (None, 64)                1600      
                                                                 
 dense_4 (Dense)             (None, 24)                1560      
                                                                 
 dense_5 (Dense)             (None, 72)                1800      
                                                                 
 dense_6 (Dense)             (None, 1)                 7

In [44]:
#now fitting data to best parametrised model..

In [46]:
model1.fit(x_train,y_train,epochs=200,initial_epoch=5,validation_data=(x_test,y_test))

Epoch 6/200
17/17 [==============================] - 1s 11ms/step - loss: 0.6688 - accuracy: 0.5531 - val_loss: 0.6292 - val_accuracy: 0.6320
Epoch 7/200
17/17 [==============================] - 0s 3ms/step - loss: 0.6072 - accuracy: 0.6797 - val_loss: 0.5998 - val_accuracy: 0.6320
Epoch 8/200
17/17 [==============================] - 0s 3ms/step - loss: 0.5815 - accuracy: 0.6667 - val_loss: 0.5653 - val_accuracy: 0.7143
Epoch 9/200
17/17 [==============================] - 0s 3ms/step - loss: 0.5652 - accuracy: 0.7076 - val_loss: 0.5619 - val_accuracy: 0.6710
Epoch 10/200
17/17 [==============================] - 0s 3ms/step - loss: 0.5383 - accuracy: 0.7281 - val_loss: 0.5242 - val_accuracy: 0.7316
Epoch 11/200
17/17 [==============================] - 0s 3ms/step - loss: 0.5215 - accuracy: 0.7393 - val_loss: 0.5921 - val_accuracy: 0.6926
Epoch 12/200
17/17 [==============================] - 0s 3ms/step - loss: 0.5197 - accuracy: 0.7430 - val_loss: 0.5030 - val_accuracy: 0.7489
Epoch 13/

In [47]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 112)               1008      
                                                                 
 dense_1 (Dense)             (None, 80)                9040      
                                                                 
 dense_2 (Dense)             (None, 24)                1944      
                                                                 
 dense_3 (Dense)             (None, 64)                1600      
                                                                 
 dense_4 (Dense)             (None, 24)                1560      
                                                                 
 dense_5 (Dense)             (None, 72)                1800      
                                                                 
 dense_6 (Dense)             (None, 1)                 7